<h1> Parameter Tunning 
    
    Look at different steps & batch sizes 
    (6 months of data used)

# Importing libraries:

In [1]:
!pip install pandas
!pip install sklearn
!pip install matplotlib

import pandas as pd;
import numpy as np;
import sklearn
import matplotlib.pyplot as plt
import time

# months to evaluate model on
date = 2018090000

# for later use
features_num = 14

# lists to append results
mae_gen = []
mae_nor = []
mae_spi = []
rmse_gen = []
rmse_nor = []
rmse_spi = []
y_pred_list = []
time_count = []

# Import data; fill nan values; split data intro train and test:

In [2]:
data = pd.read_csv('Data_set_1_smaller_(1).csv', index_col = 0)

# set predictive window according with tuning best results
data = data.loc[data.index > date, :]

# reset index
data.reset_index(inplace = True)
data.drop('index', axis = 1, inplace = True)

# fill nan values in the whole data set
data.fillna(data.mean(), inplace = True)

from sklearn.model_selection import train_test_split

# divide data into train and test 
data_train, data_test = train_test_split(
         data, test_size = 0.15, shuffle=False)

# Apply feature scaling:

In [3]:
from sklearn.preprocessing import MinMaxScaler

# data scaling
sc_X = MinMaxScaler()
data_train = sc_X.fit_transform(data_train)
data_test = sc_X.transform(data_test)

# Importing the Keras libraries and packages:

In [4]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import LeakyReLU
from keras import initializers
from keras import optimizers
from keras.callbacks import EarlyStopping

# Need to process data with spike occurences in the same way as features and offers:
(Required to evaluate predictions in both normal regions and spike regions)

In [5]:
# download data for shaded area
data = pd.read_csv('Spike_binary_1std.csv', index_col = 0)

# set predictive window according with tuning best results
data = data.loc[data.index > date, :]

# make sure shaded area will correspond to values outputed by LSTM
data.reset_index(drop = True, inplace = True)

# fill_nan is already made - so lets split data into test and train
from sklearn.model_selection import train_test_split

# divide data into train and test 
shade_train, shade_test = train_test_split(
         data, test_size = 0.15, shuffle = False)

# reset index of testing data
shade_test.reset_index(drop = True, inplace = True)

# function to split data into correct shape for RNN
def split_data_shade(shade_test, steps):
    y_spike_occ = list()
    upper_lim = list()
    lower_lim = list()
    for i in range(steps, len(shade_test.index)):
        y_spike_occ.append(shade_test['spike_occurance'][i])
        upper_lim.append(shade_test['spike_upperlim'][i])
        lower_lim.append(shade_test['spike_lowerlim'][i])
    return np.array(y_spike_occ), np.array(upper_lim), np.array(lower_lim)

# function to cut data set so it can be divisible by the batch_size
def cut_data_shade(data, batch_size):
     # see if it is divisivel
    condition = data.shape[0] % batch_size
    if condition == 0:
        return data
    else:
        return data[: -condition]

# Prepare of data according to LSTM needs,  create regressor & tune:

In [ ]:
# parameters
# steps = 96
n_hidden = 2
units = 100
# batch_size = 96

from sklearn.model_selection import ParameterGrid

# 48 - day; 96 - 2 days; 336 - 1 week
parameters = {'steps': [48, 96, 336],
              'batch_size': [48, 96, 336]}

all_param = ParameterGrid(parameters)

# function to split data into correct shape for RNN
def split_data(X, y, steps):
    X_, y_ = list(), list()
    for i in range(steps, len(y)):
        X_.append(X[i - steps : i, :])
        y_.append(y[i]) 
    return np.array(X_), np.array(y_)

# function to cut data set so it can be divisible by the batch_size
def cut_data(data, batch_size):
     # see if it is divisivel
    condition = data.shape[0] % batch_size
    if condition == 0:
        return data
    else:
        return data[: -condition]

for i in range(len(all_param)):
    
    start_time = time.time()
    
    # set parameters right
    steps = all_param[i]['steps']
    batch_size = all_param[i]['batch_size']
    
    # divide features and labels
    X_train = data_train[:, 0:14] 
    y_train = data_train[:, -1]
    X_test = data_test[:, 0:14] 
    y_test = data_test[:, -1] 

    # divide data into train and test 
    X_train, X_val, y_train, y_val = train_test_split(
             X_train, y_train, test_size = 0.15, shuffle=False)

    # put data into correct shape
    X_train, y_train = split_data(X_train, y_train, steps)
    X_test, y_test = split_data(X_test, y_test, steps)
    X_val, y_val = split_data(X_val, y_val, steps)

    X_train = cut_data(X_train, batch_size)
    y_train = cut_data(y_train, batch_size)
    X_test = cut_data(X_test, batch_size)
    y_test = cut_data(y_test, batch_size)
    X_val = cut_data(X_val, batch_size)
    y_val = cut_data(y_val, batch_size)

    def regressor_tunning(kernel_initializer = 'he_uniform',
                          bias_initializer = initializers.Ones()):
        model = Sequential()
        if n_hidden == 0:
            model.add(LSTM(units = units,                    
                           input_shape = (steps, features_num), 
                           kernel_initializer = kernel_initializer,
                           bias_initializer = bias_initializer))
            model.add(LeakyReLU(alpha = 0.2))
            model.add(Dropout(0.2))
        else:
            model.add(LSTM(units = units,                    
                           input_shape = (steps, features_num), 
                           return_sequences = True,
                           kernel_initializer = kernel_initializer,
                           bias_initializer = bias_initializer))
            model.add(LeakyReLU(alpha = 0.2))
            model.add(Dropout(0.2))
            model.add(LSTM(units = units, 
                           input_shape = (steps, features_num), 
                           kernel_initializer = kernel_initializer,
                           bias_initializer = bias_initializer))
            model.add(LeakyReLU(alpha = 0.2))
            model.add(Dropout(0.2))
        model.add(Dense(1, activation='linear'))
        optimizer = optimizers.RMSprop()
        model.compile(loss = 'mse', metrics = ['mse', 'mae'], optimizer = optimizer)
        return model

    model = regressor_tunning()

    # fitting the LSTM to the training set
    model.fit(X_train,
              y_train, 
              batch_size = batch_size, 
              epochs = 100,
              shuffle = False, 
              validation_data = (X_val, y_val))

    # reset states to have suitable predictions
    model.reset_states()
    
    # make new predicitons with test set
    y_pred = model.predict(X_test, batch_size = batch_size)
    
    # prices col = 15
    y_pred = (y_pred * sc_X.data_range_[14]) + (sc_X.data_min_[14])
    y_test = (y_test * sc_X.data_range_[14]) + (sc_X.data_min_[14])
    
    # Reshaping
    y_pred = np.reshape(y_pred, (y_pred.shape[0]))
    
    y_pred_list.append(y_pred)

    from sklearn.metrics import mean_squared_error as mse
    from sklearn.metrics import mean_absolute_error as mae

    rmse_error = mse(y_test, y_pred, squared = False)
    mae_error = mae(y_test, y_pred)

    rmse_gen.append(rmse_error)
    mae_gen.append(mae_error)

    # =============================================================================
    # Metrics evaluation on spike regions
    # =============================================================================
    
    # shape y_spike_occ for the right size to compare results in normal and spike regions
    y_spike_occ, spike_upperlim, spike_lowerlim = split_data_shade(shade_test, steps)
    y_spike_occ = cut_data_shade(y_spike_occ, batch_size)
    
    # smal adjustment
    y_test = pd.Series(y_test)
    y_test.replace(0, 0.0001,inplace = True)

    # select y_pred and y_test only for regions with spikes
    y_test_spike = (y_test.T * y_spike_occ).T
    y_pred_spike = (y_pred.T * y_spike_occ).T
    y_test_spike = y_test_spike[y_test_spike != 0]
    y_pred_spike = y_pred_spike[y_pred_spike != 0]

    # calculate metric
    rmse_spike = mse(y_test_spike, y_pred_spike, squared = False)
    mae_spike = mae(y_test_spike, y_pred_spike)

    rmse_spi.append(rmse_spike)
    mae_spi.append(mae_spike)

    # =============================================================================
    # Metric evaluation on normal regions
    # =============================================================================

    # inverse y_spike_occ so the only normal occurences are chosen
    y_normal_occ = (y_spike_occ - 1) * (-1)

    # sanity check
    y_normal_occ.sum() + y_spike_occ.sum() # gives the correct total 

    # select y_pred and y_test only for normal regions
    y_test_normal = (y_test.T * y_normal_occ).T
    y_pred_normal = (y_pred.T * y_normal_occ).T
    y_test_normal = y_test_normal[y_test_normal != 0.00]
    y_pred_normal = y_pred_normal[y_pred_normal != 0.00]

    # calculate metric
    rmse_normal = mse(y_test_normal, y_pred_normal, squared = False)
    mae_normal = mae(y_test_normal, y_pred_normal)

    rmse_nor.append(rmse_normal)
    mae_nor.append(mae_normal)

    elapsed_time = time.time() - start_time

    time_count.append(elapsed_time)


Epoch 1/100
87/87 [==============================] - 6s 69ms/step - loss: 0.1656 - mse: 0.1656 - mae: 0.2987 - val_loss: 0.0010 - val_mse: 0.0010 - val_mae: 0.0285
Epoch 2/100
87/87 [==============================] - 5s 62ms/step - loss: 0.0266 - mse: 0.0266 - mae: 0.1280 - val_loss: 0.0104 - val_mse: 0.0104 - val_mae: 0.0996
Epoch 3/100
87/87 [==============================] - 5s 63ms/step - loss: 0.0071 - mse: 0.0071 - mae: 0.0645 - val_loss: 7.8415e-04 - val_mse: 7.8415e-04 - val_mae: 0.0187
Epoch 4/100
87/87 [==============================] - 6s 64ms/step - loss: 0.0028 - mse: 0.0028 - mae: 0.0386 - val_loss: 8.5004e-04 - val_mse: 8.5004e-04 - val_mae: 0.0200
Epoch 5/100
87/87 [==============================] - 6s 64ms/step - loss: 0.0017 - mse: 0.0017 - mae: 0.0266 - val_loss: 8.3182e-04 - val_mse: 8.3182e-04 - val_mae: 0.0196
Epoch 6/100
87/87 [==============================] - 6s 68ms/step - loss: 0.0014 - mse: 0.0014 - mae: 0.0231 - val_loss: 4.9676e-04 - val_mse: 4.9676e-04 - 

87/87 [==============================] - 5s 59ms/step - loss: 9.3957e-04 - mse: 9.3957e-04 - mae: 0.0145 - val_loss: 2.9280e-04 - val_mse: 2.9280e-04 - val_mae: 0.0109
Epoch 48/100
87/87 [==============================] - 5s 60ms/step - loss: 9.1062e-04 - mse: 9.1062e-04 - mae: 0.0142 - val_loss: 2.7053e-04 - val_mse: 2.7053e-04 - val_mae: 0.0097
Epoch 49/100
87/87 [==============================] - 6s 65ms/step - loss: 9.1535e-04 - mse: 9.1535e-04 - mae: 0.0142 - val_loss: 2.8731e-04 - val_mse: 2.8731e-04 - val_mae: 0.0109
Epoch 50/100
87/87 [==============================] - 6s 71ms/step - loss: 8.7470e-04 - mse: 8.7470e-04 - mae: 0.0139 - val_loss: 2.9724e-04 - val_mse: 2.9724e-04 - val_mae: 0.0108
Epoch 51/100
87/87 [==============================] - 6s 67ms/step - loss: 9.0593e-04 - mse: 9.0593e-04 - mae: 0.0144 - val_loss: 4.7120e-04 - val_mse: 4.7120e-04 - val_mae: 0.0128
Epoch 52/100
87/87 [==============================] - 6s 68ms/step - loss: 9.0084e-04 - mse: 9.0084e-04 - ma

# Save results into a data frame

In [ ]:
# Save
results = pd.DataFrame({'rmse_general': rmse_gen, 
                 
                        'mae_general': mae_gen,
                        
                        'rmse_spike': rmse_spi,
                 
                        'mae_spike': mae_spi,
                        
                        'rmse_normal': rmse_nor,
                    
                        'mae_normal': mae_nor,
                       
                        'time': time_count})

In [ ]:
def highlight_min(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.min()
    return ['background-color: yellow' if v else '' for v in is_max]

results.style.apply(highlight_min)

In [ ]:
# Save
results = pd.DataFrame({'rmse_general': rmse_gen, 
                 
                        'mae_general': mae_gen,
                        
                        'rmse_spike': rmse_spi,
                 
                        'mae_spike': mae_spi,
                        
                        'rmse_normal': rmse_nor,
                    
                        'mae_normal': mae_nor,
                       
                        'time': time_count}, index = all_param)

results.to_csv('Results_LSTM_steps_batch.csv')

y_pred_list = pd.Series(y_pred_list)
y_pred_list.to_csv('Predictions_LSTM_steps_batch.csv')

In [ ]:
%matplotlib notebook

w_plot = 144 # 3 days
fontsize = 13

y_pred = y_pred.reshape(len(y_pred))

Residual = list(y_test) - y_pred

plt.figure(figsize=(8,3))
plt.plot(np.arange(0, (w_plot)), y_test[-w_plot:], label = 'Real values', linewidth = 1.5, color = 'steelblue')
plt.plot(np.arange(0, (w_plot)), y_pred[-w_plot:], label = 'Predicted values', linewidth = 1.2, color= 'deepskyblue')
plt.plot(np.arange(0, (w_plot)), Residual[-w_plot:], label = 'Residual error', linewidth = 0.8, color = 'slategrey')
plt.fill_between(np.arange(0, (w_plot)),  data['spike_lowerlim'][-w_plot:],data['spike_upperlim'][-w_plot:], facecolor='skyblue', alpha=0.5, label = 'Not spike regions')
plt.xlim(0, w_plot - 1)
plt.ylim(-100, 260)
plt.minorticks_on()
plt.grid(which='major', linestyle='-', linewidth='0.5')
plt.grid(which='minor', linestyle=':', linewidth='0.5')
plt.xlabel('Accumulated SP', fontsize = fontsize)
plt.ylabel('RMSE (£/MWh)', fontsize = fontsize)
plt.xticks(fontsize = fontsize)
plt.yticks([-100, -50, 0, 50,100, 150, 200, 250],[-100, -50, 0, 50, 100, 150, 200, 250],  fontsize = fontsize)
plt.title('Linear Regression predictions', fontsize = fontsize + 2)
plt.legend(loc = 'lower right')
plt.tight_layout()
plt.savefig('LSTM_1_hidden_150_nerons.png')

In [ ]:
model.summary()

In [ ]:
results